In [184]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import StandardScaler

In [185]:
RANDOM_STATE = 42

In [186]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [187]:
train_data_full = pd.read_parquet("/home/henrik/projects/cedas2025/src/data/cedas2025_material/data/chargecurves_train.parquet")

In [ ]:
len(train_data_full) / 40

In [ ]:
train_data_full.loc[train_data_full['id'] == 2]

In [190]:
def reshape_dataframe(df):
    first_timestamps = df.groupby('id')['timestamp'].first().reset_index()

    pivot_df = df.pivot(index=['id', 'nominal_power', 'location_id'],
                        columns='sub_id',
                        values=['soc', 'power']).reset_index()

    pivot_df.columns = [
        f'{col[0]}_{col[1]}' if col[1] != '' else col[0]
        for col in pivot_df.columns
    ]

    result_df = pivot_df.merge(first_timestamps, on='id')
    return result_df

In [ ]:
train_data_full = reshape_dataframe(train_data_full)
len(train_data_full)

In [ ]:
train_data_full.head()

In [ ]:
train_data_full = train_data_full.dropna()
len(train_data_full)

In [ ]:
unique_values = train_data_full['location_id'].nunique()
print(unique_values)

In [195]:
# remove rows where power_any is over 500
POWER_COLOUMNS_ALL = [f'power_{i}' for i in range(40)]
train_data_true_known = train_data_full[train_data_full[POWER_COLOUMNS_ALL].le(500).all(axis=1)]

In [196]:
# remove rows where soc_0 is 0
train_data_true_known = train_data_true_known[train_data_true_known['soc_0'] != 0]

In [ ]:
len(train_data_true_known)

In [198]:
train_data, temp_data = train_test_split(train_data_true_known,
                                         train_size=0.70,
                                         test_size=0.30,
                                         shuffle=True,
                                         random_state=RANDOM_STATE)
validation_data, test_data = train_test_split(temp_data,
                                              train_size=0.5,
                                              test_size=0.5,
                                              random_state=RANDOM_STATE)


In [199]:
train_data['month'] = train_data['timestamp'].dt.month
validation_data['month'] = validation_data['timestamp'].dt.month
test_data['month'] = test_data['timestamp'].dt.month

In [200]:
SOC_COLOUMNS_SUBSET = [f'soc_{i}' for i in range(10)]
POWER_COLOUMNS_SUBSET = [f'power_{i}' for i in range(10)]

TARGET_POWER = [f'power_{i}' for i in range(10, 40)]
TARGET_SOC = [f'soc_{i}' for i in range(10, 40)]
#----

all_columns = train_data.columns.tolist()

REMOVE = ["timestamp"]
TO_DROP_FROM_X = TARGET_POWER + TARGET_SOC + REMOVE

# Define input features by excluding the target columns
INPUT_FEATURES_CLUSTERING = [col for col in all_columns if col not in TO_DROP_FROM_X ]


In [201]:
# ONLY FIRST 10
train_data_first_ten = train_data[INPUT_FEATURES_CLUSTERING]
validation_data_first_ten = validation_data[INPUT_FEATURES_CLUSTERING]
test_data_first_ten = test_data[INPUT_FEATURES_CLUSTERING]

# train_data
train_data = train_data.drop("timestamp",axis=1)
validation_data = validation_data.drop("timestamp",axis=1)
test_data = test_data.drop("timestamp",axis=1)

In [202]:
clustering_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("cluster_model", KMeans(random_state=RANDOM_STATE, n_clusters=5))
])

In [ ]:
clustering_pipeline.fit(train_data_first_ten)

In [ ]:
labels = clustering_pipeline.named_steps["cluster_model"].labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [205]:
# feature engineer

In [ ]:
# add labels to one hot encode
train_data_first_ten['cluster'] = labels
train_data['cluster'] = labels

validation_clusters = clustering_pipeline.predict(validation_data_first_ten)
validation_data_first_ten['cluster'] = validation_clusters
validation_data['cluster'] = validation_clusters

test_clusters = clustering_pipeline.predict(test_data_first_ten)
test_data_first_ten['cluster'] = test_clusters
test_data['cluster'] = test_clusters

# Check the unique labels
unique_labels = set(labels.tolist() + validation_clusters.tolist() + test_clusters.tolist())
print("Unique cluster labels:", unique_labels)

encoder = OneHotEncoder(drop=None, sparse_output=False)
encoder.fit(train_data[['cluster']])

def one_hot_encode_and_concat(df, encoder, column_name='cluster'):
    one_hot_encoded_array = encoder.transform(df[[column_name]])

    one_hot_encoded_df = pd.DataFrame(one_hot_encoded_array,
                                      columns=[f'{column_name}_{int(cat)}' for cat in encoder.categories_[0]],
                                      index=df.index)

    df_with_one_hot = pd.concat([df, one_hot_encoded_df], axis=1)

    return df_with_one_hot


# Apply the helper function to all datasets
train_data_first_ten = one_hot_encode_and_concat(train_data_first_ten, encoder)
train_data = one_hot_encode_and_concat(train_data, encoder)

validation_data_first_ten = one_hot_encode_and_concat(validation_data_first_ten, encoder)
validation_data = one_hot_encode_and_concat(validation_data, encoder)

test_data_first_ten = one_hot_encode_and_concat(test_data_first_ten, encoder)
test_data = one_hot_encode_and_concat(test_data, encoder)

In [ ]:
def add_temp_col(df):
  df = df.copy()
  temperature_data = {
      'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
      'temperatur': [-4, -4, 0, 5, 10, 14, 17, 15, 11, 5, 1, -3]
  }
  temp_df = pd.DataFrame(temperature_data)
  df = df.merge(temp_df, on='month', how='left')
  df = df.drop(columns=['month'])
  return df


train_data_first_ten = add_temp_col(train_data_first_ten)
train_data = add_temp_col(train_data)

validation_data_first_ten = add_temp_col(validation_data_first_ten)
validation_data = add_temp_col(validation_data_first_ten)

test_data_first_ten = add_temp_col(test_data_first_ten)
test_data = add_temp_col(test_data)

In [ ]:
train_data

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(train_data_first_ten)

plt.figure(figsize=(10, 8))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, cmap='viridis', s=50)
plt.colorbar(label='Cluster Label')
plt.title('KMeans Clustering Results')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [210]:
def plot_mean():
    # Calculate the average SOC and Power for each cluster by taking the mean across all time steps
    cluster_avg_soc = train_data_first_ten.groupby('cluster')[SOC_COLOUMNS_SUBSET].mean().mean(axis=1)
    cluster_avg_power = train_data_first_ten.groupby('cluster')[POWER_COLOUMNS_SUBSET].mean().mean(axis=1)

    # Create a figure with two subplots: one for SOC and one for Power
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Bar chart for average SOC
    axes[0].bar(cluster_avg_soc.index, cluster_avg_soc.values, color='b')
    axes[0].set_title('Average SOC per Cluster')
    axes[0].set_xlabel('Cluster')
    axes[0].set_ylabel('Average SOC')
    axes[0].grid(True)

    # Bar chart for average Power
    axes[1].bar(cluster_avg_power.index, cluster_avg_power.values, color='r')
    axes[1].set_title('Average Power per Cluster')
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Average Power')
    axes[1].grid(True)

    # Show the plots
    plt.tight_layout()
    plt.show()


In [211]:
def plot_variation():
    # Combine SOC and Power columns for plotting the boxplots
    # First, we will stack the SOC columns and Power columns for each cluster

    # Create a new DataFrame to hold SOC and Power values by cluster
    soc_data = train_data_first_ten.melt(id_vars=['cluster'], value_vars=SOC_COLOUMNS_SUBSET, var_name='SOC_Timestep', value_name='SOC')
    power_data = train_data_first_ten.melt(id_vars=['cluster'], value_vars=POWER_COLOUMNS_SUBSET, var_name='Power_Timestep', value_name='Power')

    # Set up a figure for the boxplot
    plt.figure(figsize=(18, 8))

    # Plot the SOC variation
    plt.subplot(1, 2, 1)
    sns.boxplot(x='cluster', y='SOC', data=soc_data, palette='Blues')
    plt.title('SOC Variation by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('SOC')

    # Plot the Power variation
    plt.subplot(1, 2, 2)
    sns.boxplot(x='cluster', y='Power', data=power_data, palette='Reds')
    plt.title('Power Variation by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Power')

    # Show the plots
    plt.tight_layout()
    plt.show()

In [ ]:
plot_mean()
plot_variation()

In [213]:
# to_drop = ["cluster","month","id","timestamp"]

# train_data_first_ten = remove_encoded_values(train_data_first_ten, to_drop)
# train_data = remove_encoded_values(train_data, to_drop)
# validation_data_first_ten = remove_encoded_values(validation_data_first_ten, to_drop)
# validation_data = remove_encoded_values(validation_data, to_drop)
# test_data_first_ten = remove_encoded_values(test_data_first_ten, to_drop)
# test_data = remove_encoded_values(test_data, to_drop)

In [ ]:
train_data.head()

In [215]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR


def get_model_pipeline():
    return Pipeline([
        ('scaler', MinMaxScaler()),
        ('poly', PolynomialFeatures(degree=2)),
        # ('pca', PCA(n_components=0.50)),
        ('regressor', MultiOutputRegressor(Ridge())),
    ])

In [216]:
TARGET_POWER = [f'power_{i}' for i in range(10, 40)]
TARGET_SOC = [f'soc_{i}' for i in range(10, 40)]

all_columns = train_data.columns.tolist()

REMOVE = ["cluster","month","id","timestamp"]
TO_DROP_FROM_X = TARGET_POWER + TARGET_SOC + REMOVE

# Define input features by excluding the target columns
INPUT_FEATURES_MODEL = [col for col in all_columns if col not in TO_DROP_FROM_X ]


In [ ]:
train_data[INPUT_FEATURES_MODEL]

In [ ]:
models_power = {}
for label in range(0, n_clusters_):
    one_hot_column_name = f'cluster_{label}'

    train_subset = train_data[train_data[one_hot_column_name] == 1]

    X_train_lin = train_subset[INPUT_FEATURES_MODEL]
    y_train_lin_power = train_subset[TARGET_POWER]

    pipeline_model = get_model_pipeline()
    pipeline_model.fit(X_train_lin, y_train_lin_power)

    models_power[label] = pipeline_model
    print(f"Trained model for cluster {label} on {len(train_subset)} samples.")


In [ ]:
validation_scores_power = {}
for cluster, model in models_power.items():
    one_hot_column_name = f'cluster_{label}'
    val_subset = validation_data[validation_data[one_hot_column_name] == 1]

    X_val = val_subset[INPUT_FEATURES_MODEL]
    y_val = val_subset[TARGET_POWER]
    y_pred = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred)

    validation_scores_power[cluster] = mae

    print(f"Cluster {cluster}: MAE = {mae:.3f}")

In [ ]:
clusters = list(validation_scores_power.keys())
mae_values = list(validation_scores_power.values())

plt.figure(figsize=(10, 6))
plt.bar(clusters, mae_values, color='skyblue', edgecolor='black')
plt.xlabel("Cluster", fontsize=14)
plt.ylabel("Mean Absolute Error (MAE)", fontsize=14)
plt.title("Validation MAE by Cluster", fontsize=16)
plt.xticks(clusters)

for idx, mae in zip(clusters, mae_values):
    plt.text(idx, mae + 0.2, f"{mae:.2f}", ha="center", va="bottom", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
total_instances = len(validation_data)
weighted_sum = 0

for cluster, mae in validation_scores_power.items():
    one_hot_column_name = f'cluster_{cluster}'
    count = len(validation_data[validation_data[one_hot_column_name] == 1])
    weighted_sum += mae * count
    print(f"Cluster {cluster}: instances = {count}, MAE = {mae:.3f}")

weighted_avg_mae = weighted_sum / total_instances
print(f"\nTotal instances in validation data: {total_instances}")
print(f"Weighted Average MAE POWER: {weighted_avg_mae:.3f}")

In [122]:
# SOC IS NEXT
TARGET_SOC = [f'soc_{i}' for i in range(10, 40)]

In [ ]:
models_soc = {}
for label in range(0, n_clusters_):
    one_hot_column_name = f'cluster_{label}'
    train_subset = train_data[train_data[one_hot_column_name] == 1]

    X_train_lin = train_subset[INPUT_FEATURES_MODEL]
    y_train_lin_power = train_subset[TARGET_SOC]

    pipeline_model = get_model_pipeline()
    pipeline_model.fit(X_train_lin, y_train_lin_power)

    models_soc[label] = pipeline_model
    print(f"Trained model for cluster {label} on {len(train_subset)} samples.")


In [ ]:
validation_scores_soc = {}
for cluster, model in models_soc.items():
    one_hot_column_name = f'cluster_{label}'
    val_subset = validation_data[validation_data[one_hot_column_name] == 1]

    X_val = val_subset[INPUT_FEATURES_MODEL]
    y_val = val_subset[TARGET_SOC]

    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    validation_scores_soc[cluster] = mae

    print(f"Cluster {cluster}: MAE = {mae:.3f}")

In [ ]:
for cluster, model in models_soc.items():
    one_hot_column_name = f'cluster_{label}'
    val_subset = validation_data[validation_data[one_hot_column_name] == 1]
    
    X_val = val_subset[INPUT_FEATURES_MODEL]
    y_val = val_subset[TARGET_SOC]

    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    validation_scores_soc[cluster] = mae

    print(f"Cluster {cluster}: MAE = {mae:.3f}")

In [ ]:
clusters = list(validation_scores_soc.keys())
mae_values = list(validation_scores_soc.values())

plt.figure(figsize=(10, 6))
plt.bar(clusters, mae_values, color='skyblue', edgecolor='black')
plt.xlabel("Cluster", fontsize=14)
plt.ylabel("Mean Absolute Error (MAE)", fontsize=14)
plt.title("Validation MAE by Cluster", fontsize=16)
plt.xticks(clusters)

for idx, mae in zip(clusters, mae_values):
    plt.text(idx, mae + 0.2, f"{mae:.2f}", ha="center", va="bottom", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
total_instances = len(validation_data)
weighted_sum = 0

for cluster, mae in validation_scores_soc.items():
    one_hot_column_name = f'cluster_{cluster}'
    count = len(validation_data[validation_data[one_hot_column_name] == 1])
    weighted_sum += mae * count
    print(f"Cluster {cluster}: instances = {count}, MAE = {mae:.3f}")

weighted_avg_mae = weighted_sum / total_instances
print(f"\nTotal instances in validation data: {total_instances}")
print(f"Weighted Average MAE SOC: {weighted_avg_mae:.3f}")